In [3]:
from mistle_v2 import get_dl, Mistle, Theory
from time import time
from data_generators import TheoryNoisyGeneratorOnDataset, GeneratedTheory
import random
import numpy as np
import pycosat

In [13]:
seed = 0
random.seed(seed)
np.random.seed(seed)

start_time = time()

th = GeneratedTheory([[1, -4], [2, 5], [6, -7, -8]])
generator = TheoryNoisyGeneratorOnDataset(th, 200, 0.7)
positives, negatives = generator.generate_dataset()


mistle = Mistle(positives, negatives)
theory, compression = mistle.learn(minsup=1, dl_measure="ce")
print("Total time\t\t\t\t: " + str(time() - start_time) + " seconds.")
if theory is not None:
    print("Final theory has " + str(len(theory.clauses)) + " clauses.")
else:
    print("Empty theory learned.")
    

Length of freq items 1	: 29
Time of freq items 1	: 0.22682976722717285
124 Frequent Itemsets are mined in 0.0010025501251220703 seconds.
Length of freq items 2	: 124
Time of freq items 2	: 0.001995086669921875
DL of initial theory	: 2047.7712614179425
10	frequent itemsets left after applying S operator with output 	: [frozenset({1, -4})]
4	frequent itemsets left after applying S operator with output 	: [frozenset({2, 3, 5})]
1	frequent itemsets left after applying T operator with output 	: [frozenset({2, 5})]
Length of freq items 1	: 4
Time of freq items 1	: 0.24328207969665527
19 Frequent Itemsets are mined in 0.0 seconds.
Length of freq items 2	: 19
Time of freq items 2	: 0.0
0 clauses are pruned.
Initial DL				: 2571.6596837179623
Final DL				: 34.981428833018086
Compression (wrt ce)	: 0.9863973335762515
Operator Counters		: {'W': 0, 'V': 0, 'S': 2, 'R': 0, 'T': 1}
Total time				: 0.5060055255889893 seconds.
Final theory has 3 clauses.


In [11]:
print([list(c) for c in theory.clauses])

[[2, 5], [1, -4], [-8, -7, 6]]


In [5]:
nb_models = {}
clauses = [frozenset([1, -4]), frozenset([2, 5]), frozenset([6, -7, -8]),
           frozenset([-1, -3, -4, 5, -6, 7, -8]),frozenset([-1, 2, -4, -5, 6, -8]),frozenset([-1, 2, 3, -4, -5, 6, -7])]
#clauses = theory.unpack_theory()
for c in clauses:
    new_theory = list(clauses)
    new_theory.remove(c)
    nb_models[c] = len(list(pycosat.itersolve(new_theory)))

nb_models_ref= len(list(pycosat.itersolve(clauses)))
print(nb_models)

{frozenset({1, -4}): 164, frozenset({2, 5}): 163, frozenset({-8, -7, 6}): 138, frozenset({5, 7, -8, -6, -4, -3, -1}): 123, frozenset({2, 6, -8, -5, -4, -1}): 124, frozenset({2, 3, 6, -7, -5, -4, -1}): 123}


In [4]:
new_errors = {}
new_errors_n = {}
new_errors_total = {}
clauses = [frozenset([1, -4]), frozenset([2, 5]), frozenset([6, -7, -8]),
           frozenset([-1, -3, -4, 5, -6, 7, -8]),frozenset([-1, 2, -4, -5, 6, -8]),frozenset([-1, 2, 3, -4, -5, 6, -7])]
#clauses = theory.unpack_theory()
for c in clauses:
    new_theory = list(clauses)
    new_theory.remove(c)
    m_theory = Theory(new_theory, None, None)
    violations = m_theory.get_violations(m_theory.clauses, positives, print_violations=False)
    violations_n = m_theory.get_violations(m_theory.clauses, negatives, print_violations=False, sign="-")
    new_errors[c] = violations
    new_errors_n[c] = violations_n
    new_errors_total[c] = violations.union(violations_n)

ref_theory = Theory(clauses, None, None)
ref_error = ref_theory.get_violations(ref_theory.clauses, positives, print_violations=False)
ref_error_n = ref_theory.get_violations(ref_theory.clauses, negatives, print_violations=False, sign="-")
ref_error_total = ref_error.union(ref_error_n)

In [30]:
for n in negatives:
    if frozenset([-7, -2, -8, -6, -5, -4, -3]) <= n:
        print(n)


frozenset({1, -8, -7, -6, -5, -4, -3, -2})
frozenset({1, -8, -7, -6, -5, -4, -3, -2})
frozenset({-2, -8, -7, -6, -5, -4, -3, -1})
frozenset({1, -8, -7, -6, -5, -4, -3, -2})
frozenset({-2, -8, -7, -6, -5, -4, -3, -1})
